# **Módulo 3: Introducción a PyTorch**

## Introducción
[Pytorch](https://pytorch.org/) es una librería de tensores especializados para *deep learning* que utiliza CPU y GPU. Los tensores son estructuras de datos especializadas, muy similares a los *arrays* y las matrices. Se parecen a los *arrays* de NumPy, salvo porque los tensores se pueden ejecutar en una GPU u otros aceleradores de *hardware*. En el entrenamiento de modelos, las GPU ofrecen una capacidad de computación mucho más rápida y eficiente que la de las CPU. Además, los tensores incluyen una serie de cálculos de gradiente integrados que simplican el código en gran medida. 

## Tensores

In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt

Estos son tensores creados directamente a partir de una lista. El tipo de datos se deduce automáticamente.

In [2]:
data = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
tensor_data = torch.tensor(data)
print(tensor_data)

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])


# Este es un tensor creado a partir de un *array* de NumPy.

In [3]:
np_array = np.array(data)
tensor_np = torch.from_numpy(np_array)
print(tensor_np)

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])


Se pueden crear nuevos tensores a partir de otros. El nuevo tensor retiene las propiedades (forma, tipo de datos) del tensor argumento, a menos que se especifique lo contrario.

In [4]:
x_zeros = torch.zeros_like(tensor_data) # Retener las propiedades de tensor_data
print(f"Zeros Tensor: \n {x_zeros} \n")

x_ones = torch.ones_like(tensor_data) # Retener las propiedades de tensor_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(tensor_data, dtype=torch.float) # Anular los tipos de datos de tensor_data
print(f"Random Tensor: \n {x_rand} \n")

Zeros Tensor: 
 tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]) 

Ones Tensor: 
 tensor([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]]) 

Random Tensor: 
 tensor([[0.3097, 0.9700, 0.5764],
        [0.2597, 0.0588, 0.6836],
        [0.2000, 0.3109, 0.4028]]) 



También se puede crear un tensor a partir de valores aleatorios o constantes.
En la función que aparece más abajo, añadimos una tupla, la forma, que especifica la dimensión del tensor de *output*.

In [5]:
shape = (3, 2,)
zeros_tensor = torch.zeros(shape)
ones_tensor = torch.ones(shape)
rand_tensor = torch.rand(shape)


print(f"Zeros Tensor: \n {zeros_tensor}")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Random Tensor: \n {rand_tensor} \n")

Zeros Tensor: 
 tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])
Ones Tensor: 
 tensor([[1., 1.],
        [1., 1.],
        [1., 1.]]) 

Random Tensor: 
 tensor([[0.6332, 0.7867],
        [0.4947, 0.7626],
        [0.5672, 0.3837]]) 



Ademas de los conceptos de *forma* y *tipo de datos*; vamos a introducir un concepto nuevo, conocido como **dispositivo**, que especifica dónde se ejecutan los cálculos (hardware). Las GPU tienen un número mayor de unidades aritméticas lógicas o ALU, unidades de control y memoria caché cuya función básica es procesar en paralelo series de cálculos simples e idénticos.

In [6]:
print(f"Shape of tensor: {rand_tensor.shape}")
print(f"Datatype of tensor: {rand_tensor.dtype}")
print(f"Device tensor is stored on: {rand_tensor.device}")

Shape of tensor: torch.Size([3, 2])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


Nvidia creó CUDA, una plataforma y arquitectura de computación en paralelo para sus GPU. El código que aparece más abajo cambiará el dispositivo de la CPU a la GPU, si hay una disponible.

In [7]:
if torch.cuda.is_available():
  tensor = rand_tensor.to('cuda')
  print(f"Device tensor is stored on: {tensor.device}")

Device tensor is stored on: cuda:0


## Gradientes

El siguiente ejemplo está extraído de: https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html

El algoritmo más utilizado para entrenar redes neuronales es la retropropagación. En este algoritmo, se ajustan los parámetros (pesos del modelo) en función del gradiente de la función de pérdida respecto al parámetro dado.

Para computar estos gradientes, PyTorch cuenta con un motor de diferenciación integrado llamado "torch.autograd", que soporta la computación automática del gradiente de cualquier gráfico computacional.

Consideremos la red neuronal más sencilla de una capa, con un *input* x, parámetros w y b, y una función de pérdida. Se puede definir en PyTorch de la siguiente manera:

In [13]:
import torch

x = torch.ones(5)  # Tensor de input
y = torch.zeros(3)  # Output esperado
# Shape of w: [5, 3]
w = torch.randn(5, 3, requires_grad=True)   # Hay que computar el gradiente respecto a este parametro
b = torch.randn(3, requires_grad=True)      # para optimizar la funcion de perdida
z = torch.matmul(x, w)+b

# Pérdida de Log-Verosimilitud Negativa (NLL)
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

print("loss:", loss)

loss: tensor(1.4710, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


Una función que aplicamos a los tensores para construir gráficos computacionales es de hecho un objeto de la clase Función. Este objeto sabe cómo computar la función hacia delante, y también cómo computar su derivada durante el paso de retropropagación. En la propiedad grad_fn de un tensor se almacena una referencia a la función de retropropagación. Puede encontrar más información al respecto en la documentación.

In [14]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7f69a57e8a60>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7f699ced7580>


Para optimizar los pesos de los parámetros de la red neuronal, tenemos que computar las derivadas de la función de pérdida respecto a los parámetros; concretamente, necesitamos $\frac{\partial loss}{\partial w}$
  y $\frac{\partial loss}{\partial b}$ 
  con unos valores fijos de x e y. Para computar esas derivadas, llamamos a loss.backward(), y después recuperamos los valores de w.grad y b.grad:

In [10]:
loss.backward() # Calcular los gradientes
print(w.grad)
print(b.grad)

tensor([[0.3163, 0.0084, 0.2742],
        [0.3163, 0.0084, 0.2742],
        [0.3163, 0.0084, 0.2742],
        [0.3163, 0.0084, 0.2742],
        [0.3163, 0.0084, 0.2742]])
tensor([0.3163, 0.0084, 0.2742])


## Optimización

Al utilizar las funciones de autodiferenciación de PyTorch, podemos realizar fácilmente las tareas de optimización necesarias para entrenar grandes redes neuronales. El fragmento de código que aparece más abajo nos permite trazar una función y su derivada correspondiente para visualizarlas.

En *deep learning*, nuestro objetivo es minimizar la función de pérdida y, en este sencillo gráfico, vemos cómo la derivada nos puede informar del valor mínimo de una función dada.

Fijémonos en el uso de .detach.numpy() para recuperar un valor del tensor y del dispositivo e introducirlo en un *array* de NumPy que lo trace.

In [5]:
# Computar la derivada de la función con múltiples valores
x = torch.linspace(-4, 4, 21, requires_grad = True)
Y = 5*x ** 2
y = torch.sum(Y)
print(y)
# y.backward()
 
# Trazar la función y la derivada
# ! detach: remover los grafos computacionales del tensor para pasarlo a numpy
# plt.plot(x.detach().numpy(), Y.detach().numpy(), label = 'Function', color='r')
# plt.plot(x.detach().numpy(), x.grad.detach().numpy(), label = 'Derivative', color='g')

# plt.xlabel('x')
# plt.legend()
# plt.grid(True)
# plt.show()

tensor(616., grad_fn=<SumBackward0>)


## ¿Qué viene después?

Como hemos visto, los tensores pueden ser una herramienta muy potente y simplificar en gran medida la implementación de los programas. En los siguientes módulos, seguiremos construyendo sobre este marco de autogradiente y lo aplicaremos a redes neuronales más complejas. Para obtener más información sobre los tensores y sobre PyTorch, consulte el tutorial y los documentos que encontrará [aquí](https://pytorch.org/tutorials/beginner/basics/intro.html).